# Indicator synthesis

In [ ]:
%run ../notebook_preamble.ipy

import seaborn as sn
from eis.utils.data_processing import *

In [ ]:
country_codes = pd.read_csv(
    'https://www.eea.europa.eu/data-and-maps/data/waterbase-lakes-4/country-codes-and-abbreviations-32-records/country-codes-and-abbreviations-32-records/at_download/file')

eu_codes = set([x.lower() for x in country_codes['ISO2']])

with open(f"{project_dir}/data/aux/all_clean_names.json",'r') as infile:
    
    clean_names= json.load(infile)
    
with open(f"{project_dir}/data/aux/eurostat_clean_names.json",'r') as infile:
    
    es_clean_names= json.load(infile)

In [ ]:
all_names = {**clean_names,**es_clean_names}

## Load data

In [ ]:
#Create a unified indicator table
inds = pd.concat([pd.read_csv(f"{project_dir}/data/processed/{name}_indicators.csv") for name in
                 ['official','web']])

inds['country'] = [x.lower() for x in inds['country']]

inds['variable_clean'] = [all_names[x] for x in inds['variable']]

inds['country'] = ['gb' if x=='uk' else x for x in inds['country']]

In [ ]:
inds.head()

In [ ]:
#Correlation

inds_2018 = inds.loc[inds['year']==2018].pivot_table(
    index='country',columns='variable',values='value').loc[eu_codes]

inds_2018

In [ ]:
ax = sn.pairplot(inds_2018,x_vars=[x for x in ['tech_meetups','python_downloads','github_users'] if x in inds_2018.columns],
           y_vars=[x for x in list(es_clean_names.keys()) if x in inds_2018.columns],
           height=1.5,aspect=1.8)
plt.tight_layout()    

save_fig('fig_15_comparison.pdf',material_outputs)

In [ ]:
country_ranks = inds_2018.rank()

country_positions = country_ranks.mean(axis=1).sort_values(ascending=False).index

fig,ax = plt.subplots(figsize=(8,9))

sn.heatmap(country_ranks.loc[country_positions,[x for x in list(clean_names.keys()) 
                                               if x in country_ranks.columns]],cmap='Purples',ax=ax)

ax.set_xticklabels([clean_names[x.get_text()][:30]+'...' for x in ax.get_xticklabels()])
ax.set_yticklabels([x.get_text() for x in ax.get_yticklabels()],rotation=0)

plt.tight_layout()

save_fig('fig_16_country_heatmap.pdf',material_outputs)
